In [1]:
import pandas as pd
import sqlite3 as db
from langdetect import detect
import chess
import chess.pgn
import io
import utils

In [2]:
con = db.connect("./chess.db")
games = pd.read_sql_query("SELECT * FROM gameknot_games", con)
games

,id,pgn,language
0,44,"[Event ""alygally&#039;s mini-tournament""]\n[Da...",en
1,62,"[Event ""League division D2""]\n[Date ""08-Jun-06...",en
2,64,"[Event ""Let&#039;s play chess""]\n[Date ""25-May...",en
3,66,"[Event ""easy street""]\n[Date ""26-Jun-06""]\n[Wh...",en
4,70,"[Event ""001 mini-tournament""]\n[Date ""31-May-0...",en
...,...,...,...
12736,70354,"[Event ""A question of queens...""]\n[Date ""14 S...",en
12737,70355,"[Event ""50th World Open U1800 Round 9 '22""]\n[...",en
12738,70359,"[Event ""Team match""]\n[Date ""2022.06.26""]\n[Wh...",en
12739,70364,"[Event ""Team match""]\n[Date ""2022.06.30""]\n[Wh...",en


In [3]:
games.head(3)

,id,pgn,language
0,44,"[Event ""alygally&#039;s mini-tournament""]\n[Da...",en
1,62,"[Event ""League division D2""]\n[Date ""08-Jun-06...",en
2,64,"[Event ""Let&#039;s play chess""]\n[Date ""25-May...",en


In [15]:
game = chess.pgn.read_game(io.StringIO(games.pgn[0]))

In [21]:
for node in game.mainline():
    print(node.parent.board())
    print(node.move, node.ply())
    print("-----------")

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
e2e4 1
-----------
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
e7e5 2
-----------
r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
b1c3 3
-----------
r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. . . . P . . .
. . N . . . . .
P P P P . P P P
R . B Q K B N R
f8c5 4
-----------
r n b q k . n r
p p p p . p p p
. . . . . . . .
. . b . p . . .
. . . . P . . .
. . N . . . . .
P P P P . P P P
R . B Q K B N R
f1c4 5
-----------
r n b q k . n r
p p p p . p p p
. . . . . . . .
. . b . p . . .
. . B . P . . .
. . N . . . . .
P P P P . P P P
R . B Q K . N R
g8f6 6
-----------
r n b q k . . r
p p p p . p p p
. . . . . n . .
. . b . p . . .
. . B . P . . .
. . N . . . . .
P P P P . P P P
R . B 